In [ ]:
pip install kaggle

# **Import Dataset from Kaggle**

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"mdrohanislam","key":"3c3798f6e5783963f996caa565a7b4b1"}'}

# **Downloading The Dataset**

In [ ]:
!pip install kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d shawontanvir/bengali-fake-review-dataset

# Unzip it
!unzip bengali-fake-review-dataset.zip -d ./bengali_reviews

Dataset URL: https://www.kaggle.com/datasets/shawontanvir/bengali-fake-review-dataset
License(s): Attribution-NonCommercial-NoDerivatives 4.0 International (CC BY-NC-ND 4.0)
bengali-fake-review-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)
Archive:  bengali-fake-review-dataset.zip
replace ./bengali_reviews/fake.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# **Load Dataset**

In [ ]:
import pandas as pd

fake_df = pd.read_csv("bengali_reviews/fake.csv")
fake_df.head()

,Review,Label
0,।।।।।।।।।।।।Buy one get one Mexican পাস্তা।।।...,0
1,।।#review দেখে গেছিলাম\nভাবতেই অবাক লাগে ১২ রক...,0
2,#Partner_Platter ❤\n#বিগ_বাইট রেষ্টুরেন্ট এবার...,0
3,অথেনটিক পিজ্জা খেতে চাইলে এইটা মাস্ট ট্রাই ❤️\...,0
4,অনলি ফর ইউনিভার্সিটি কাপল স্টুডেন্ট \nধানমন্ডি...,0


In [ ]:
non_fake_df = pd.read_csv("bengali_reviews/non-fake.csv")
non_fake_df.head()

,Review,Label
0,অকা...অবশেষে এত রিভিউ দেখে খাইতে গেলাম #Popeye...,1
1,"***অকি! কুনো লঞ্চ,বাসের টিকিট ছাড়াই আইজ মুই বর...",1
2,Octopus Madness De Flame বনানীতে পেয়ে গেলাম মন...,1
3,অক্টোপাস ফ্রাই...দিল্লি দরবার...বসুন্ধরা সিটি ...,1
4,Autograph Live Music Cafe (বনশ্রী ) আপনি নারু...,1


# **Combine and Label**

In [ ]:
import pandas as pd

# Load and label
fake_df = pd.read_csv("bengali_reviews/fake.csv")
non_fake_df = pd.read_csv("bengali_reviews/non-fake.csv")

fake_df['label'] = 0
non_fake_df['label'] = 1

# Combine and shuffle
df = pd.concat([fake_df, non_fake_df], axis=0).sample(frac=1, random_state=42).reset_index(drop=True)


# **Split Data**

In [ ]:
from sklearn.model_selection import train_test_split

X = df['Review']  # Text reviews
y = df['label']   # Labels: 0 = Fake, 1 = Non-Fake

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)


# **Preprocess & Tokenize**

In [ ]:
import tensorflow as tf
from transformers import TFBertForSequenceClassification, BertTokenizer
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy

# Assuming 'tokenizer' is defined elsewhere
tokenizer = BertTokenizer.from_pretrained("sagorsarker/bangla-bert-base")  # Load tokenizer


model = TFBertForSequenceClassification.from_pretrained(
    "sagorsarker/bangla-bert-base",
    num_labels=2
)

# Use SparseCategoricalCrossentropy for binary classification with integer labels
model.compile(
    optimizer='adam',
    loss=SparseCategoricalCrossentropy(from_logits=True),  # Explicit loss function
    metrics=['accuracy']
)

from sklearn.utils import class_weight
import numpy as np

# Calculate class weights
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights_dict = {i: w for i, w in enumerate(class_weights)}

# Later, during model.fit:
model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=5,
    class_weight=class_weights_dict
)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/5
  4/453 [..............................] - ETA: 4:05:11 - loss: 1.3482 - accuracy: 0.6562

In [ ]:
from sklearn.metrics import classification_report

# Get predictions on validation set
y_pred_logits = model.predict(val_dataset).logits
y_pred = np.argmax(y_pred_logits, axis=1)

# Print classification report
print(classification_report(y_val, y_pred, digits=4))


# **Evaluate**

In [ ]:
loss, accuracy = model.evaluate(val_dataset)
print(f"Validation Accuracy: {accuracy:.4f}")

114/114 [==============================] - 20s 170ms/step - loss: 0.4819 - accuracy: 0.8519
Validation Accuracy: 0.8519


In [ ]:
# ✅ Example new review (Bengali text)
new_review = "এই রেস্টুরেন্টের খাবার একদমই ভালো না। রিভিউ দেখে ভুল করেছিলাম।"

# ✅ Tokenize the review just like you did during training
new_review_encoding = tokenizer(
    [new_review],                          # Input must be a list
    truncation=True,                       # Cut off if longer than max_length
    padding=True,                          # Pad shorter sequences
    max_length=150,                        # Same max length as training
    return_tensors="tf"                    # Return as TensorFlow tensors
)

# ✅ Pass the encoded input to the trained model
prediction = model(new_review_encoding)

# ✅ Extract the predicted class (0 or 1)
predicted_label = tf.argmax(prediction.logits, axis=1).numpy()[0]

# ✅ Map prediction to human-readable label
if predicted_label == 0:
    print("🔴 The review is predicted to be **Fake**.")
else:
    print("🟢 The review is predicted to be **Non-Fake**.")


🟢 The review is predicted to be **Non-Fake**.


In [ ]:
test_reviews = [
    "এই রেস্টুরেন্টের খাবার একদমই ভালো না। রিভিউ দেখে ভুল করেছিলাম।",
    "চমৎকার সার্ভিস এবং পরিবেশ। আবার আসব ইনশাআল্লাহ।",
    "ভুয়া রিভিউ দিয়ে মানুষকে প্রতারিত করা হচ্ছে।",
    "খুব খারাপ অভিজ্ঞতা। কখনোই আবার এখানে যাব না।",
    "খুবই সুস্বাদু খাবার ছিল, বিশেষ করে বিরিয়ানি।",
    "সব কিছুই অসাধারণ ছিল। আমার পরিবারের সবাই খুশি।",
    "এটা সম্পূর্ণ ভুয়া রিভিউ। আসলে জায়গাটা একদম বাজে।",
    "তারা অনেক আন্তরিক ছিল এবং খাবার সময়মত দিয়েছে।",
    "এই রেস্টুরেন্ট এক নম্বর প্রতারক। একদম বিশ্বাস করবেন না।",
    "দারুন জায়গা! পরবর্তী ভ্রমণে আবারও যাব।"
]


In [ ]:
for i, review in enumerate(test_reviews):
    encoded = tokenizer(
        [review],
        truncation=True,
        padding=True,
        max_length=150,
        return_tensors="tf"
    )

    prediction = model(encoded)
    predicted_label = tf.argmax(prediction.logits, axis=1).numpy()[0]

    label_text = "🔴 Fake" if predicted_label == 0 else "🟢 Non-Fake"

    print(f"{i+1}. {review}")
    print(f"   ➤ Prediction: {label_text}\n")


1. এই রেস্টুরেন্টের খাবার একদমই ভালো না। রিভিউ দেখে ভুল করেছিলাম।
   ➤ Prediction: 🟢 Non-Fake

2. চমৎকার সার্ভিস এবং পরিবেশ। আবার আসব ইনশাআল্লাহ।
   ➤ Prediction: 🟢 Non-Fake

3. ভুয়া রিভিউ দিয়ে মানুষকে প্রতারিত করা হচ্ছে।
   ➤ Prediction: 🟢 Non-Fake

4. খুব খারাপ অভিজ্ঞতা। কখনোই আবার এখানে যাব না।
   ➤ Prediction: 🟢 Non-Fake

5. খুবই সুস্বাদু খাবার ছিল, বিশেষ করে বিরিয়ানি।
   ➤ Prediction: 🟢 Non-Fake

6. সব কিছুই অসাধারণ ছিল। আমার পরিবারের সবাই খুশি।
   ➤ Prediction: 🟢 Non-Fake

7. এটা সম্পূর্ণ ভুয়া রিভিউ। আসলে জায়গাটা একদম বাজে।
   ➤ Prediction: 🟢 Non-Fake

8. তারা অনেক আন্তরিক ছিল এবং খাবার সময়মত দিয়েছে।
   ➤ Prediction: 🟢 Non-Fake

9. এই রেস্টুরেন্ট এক নম্বর প্রতারক। একদম বিশ্বাস করবেন না।
   ➤ Prediction: 🟢 Non-Fake

10. দারুন জায়গা! পরবর্তী ভ্রমণে আবারও যাব।
   ➤ Prediction: 🟢 Non-Fake



In [ ]:
fake_test_reviews = [
    "এই রেস্টুরেন্টে জীবনে একবার গেলেই হবে। খাবার এতটাই খারাপ যে ভাষায় বোঝানো যাবে না। অথচ রেটিং ৫ স্টার!",
    "খুব ভালো খাবার, দারুন পরিবেশ, ১০০% সুপার! (আসলে আমি কখনো যাইনি)",
    "সেরা হোটেল! আমার ভাইয়ের হোটেল তাই ৫ স্টার দিতেই হবে।",
    "খাবার খারাপ ছিল, কিন্তু রেটিং ৫ দিলাম যাতে ওদের লাভ হয়।",
    "এই রেস্টুরেন্টে না গিয়েই রিভিউ দিচ্ছি কারণ আমার বন্ধুরা বলেছে ভালো।",
    "অত্যন্ত বাজে অভিজ্ঞতা, সবাই মিথ্যা বলছে ভালো রিভিউ দিয়ে।",
    "খুব খারাপ, খুব খারাপ, একদম খারাপ... (রিভিউ কপি-পেস্ট করা)",
    "আমি ওদের প্রতিযোগী, তাই ১ স্টার দিচ্ছি যেন ব্যবসা কমে।",
    "এই রেস্টুরেন্টে যাওয়া মানেই টাকা নষ্ট। রিভিউ দেখে কেউ প্রতারিত হবেন না।",
    "এখানে কেউ খেতে আসে না, সব রিভিউ ফেক। আমি তো কখনো দেখিইনি কাস্টমার।"
]


In [ ]:
for i, review in enumerate(fake_test_reviews):
    encoded = tokenizer(
        [review],
        truncation=True,
        padding=True,
        max_length=150,
        return_tensors="tf"
    )

    prediction = model(encoded)
    predicted_label = tf.argmax(prediction.logits, axis=1).numpy()[0]

    label_text = "🔴 Fake" if predicted_label == 0 else "🟢 Non-Fake"

    print(f"{i+1}. {review}")
    print(f"   ➤ Prediction: {label_text}\n")


1. এই রেস্টুরেন্টে জীবনে একবার গেলেই হবে। খাবার এতটাই খারাপ যে ভাষায় বোঝানো যাবে না। অথচ রেটিং ৫ স্টার!
   ➤ Prediction: 🟢 Non-Fake

2. খুব ভালো খাবার, দারুন পরিবেশ, ১০০% সুপার! (আসলে আমি কখনো যাইনি)
   ➤ Prediction: 🟢 Non-Fake

3. সেরা হোটেল! আমার ভাইয়ের হোটেল তাই ৫ স্টার দিতেই হবে।
   ➤ Prediction: 🟢 Non-Fake

4. খাবার খারাপ ছিল, কিন্তু রেটিং ৫ দিলাম যাতে ওদের লাভ হয়।
   ➤ Prediction: 🟢 Non-Fake

5. এই রেস্টুরেন্টে না গিয়েই রিভিউ দিচ্ছি কারণ আমার বন্ধুরা বলেছে ভালো।
   ➤ Prediction: 🟢 Non-Fake

6. অত্যন্ত বাজে অভিজ্ঞতা, সবাই মিথ্যা বলছে ভালো রিভিউ দিয়ে।
   ➤ Prediction: 🟢 Non-Fake

7. খুব খারাপ, খুব খারাপ, একদম খারাপ... (রিভিউ কপি-পেস্ট করা)
   ➤ Prediction: 🟢 Non-Fake

8. আমি ওদের প্রতিযোগী, তাই ১ স্টার দিচ্ছি যেন ব্যবসা কমে।
   ➤ Prediction: 🟢 Non-Fake

9. এই রেস্টুরেন্টে যাওয়া মানেই টাকা নষ্ট। রিভিউ দেখে কেউ প্রতারিত হবেন না।
   ➤ Prediction: 🟢 Non-Fake

10. এখানে কেউ খেতে আসে না, সব রিভিউ ফেক। আমি তো কখনো দেখিইনি কাস্টমার।
   ➤ Prediction: 🟢 Non-Fake



In [ ]:
df['label'].value_counts()


,count
label,
1,7710
0,1339


# **Save the Model**

In [ ]:
model.save_pretrained("./bangla_bert_fake_review_model")
tokenizer.save_pretrained("./bangla_bert_fake_review_model")


Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabulary indices are not consecutive. Please check that the vocabulary is not corrupted!
Saving vocabulary to ./bangla_bert_fake_review_model/vocab.txt: vocabu

('./bangla_bert_fake_review_model/tokenizer_config.json',
 './bangla_bert_fake_review_model/special_tokens_map.json',
 './bangla_bert_fake_review_model/vocab.txt',
 './bangla_bert_fake_review_model/added_tokens.json')